## Swwet list taxi - series numéricas

## 1. Preparar datos 

In [3]:
#importar librerias
import pandas as pd

In [7]:
#cargar datos en un dataframe
df_taxis = pd.read_csv('/Users/whitneyrios/PYTHON/My projects py/proyecto 13/Sweet-Lift-Taxi--Series-numericas/dataset/taxi.csv', parse_dates=[0], index_col=[0])

In [8]:
#visualizar los datos
df_taxis.head()

,num_orders
datetime,
2018-03-01 00:00:00,9
2018-03-01 00:10:00,14
2018-03-01 00:20:00,28
2018-03-01 00:30:00,20
2018-03-01 00:40:00,32


In [9]:
#viendo la informacion general 
df_taxis.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 26496 entries, 2018-03-01 00:00:00 to 2018-08-31 23:50:00
Data columns (total 1 columns):
 #   Column      Non-Null Count  Dtype
---  ------      --------------  -----
 0   num_orders  26496 non-null  int64
dtypes: int64(1)
memory usage: 414.0 KB
